In [ ]:
### IMPORTS ###
import pandas as pd
import numpy as np
import pathlib as pl
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.metrics import classification_report

!pip install datasets
from datasets.dataset_dict import DatasetDict
from datasets import Dataset

!pip install transformers
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import get_scheduler
from transformers import TrainingArguments

import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW

!pip install evaluate
import evaluate

from transformers import Trainer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

PATH = "/content/drive/My Drive/PLN/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
### PARTICIÓN ###
df = pd.read_csv(PATH + "train.csv",  sep=',', on_bad_lines='skip', encoding='utf-8', encoding_errors='ignore', index_col=False)
df = df[['tweet', 'prejudice_lgbtiq']]
df = df.rename(columns={"tweet": "Text", "prejudice_lgbtiq": "Label"})
df.fillna(" ", inplace=True)

X_train = df['Text']
y_train = df['Label']

X_train, X_aux, y_train, y_aux = train_test_split(X_train, y_train, test_size=0.3, random_state=55, stratify=y_train)
X_val, X_test, y_val, y_test = train_test_split(X_aux, y_aux, test_size=0.66, random_state=55, stratify=y_aux)

X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)

print('Tamaño conjunto de Entrenamiento:', len(X_train))
print('Tamaño conjunto de Validación:', len(X_val))
print('Tamaño conjunto de Evaluación:', len(X_test))

d = {'train':Dataset.from_dict({'Label':y_train, 'Text':X_train}),
     'val':Dataset.from_dict({'Label':y_val, 'Text':X_val}),
     'test':Dataset.from_dict({'Label':y_test, 'Text':X_test})
     }

dict_dataset = DatasetDict(d)

Tamaño conjunto de Entrenamiento: 1869
Tamaño conjunto de Validación: 272
Tamaño conjunto de Evaluación: 530


# Tokenization

In [ ]:
model_name ='dccuchile/distilbert-base-spanish-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

MAX_LENGTH = 55

In [ ]:
def tokenize(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=MAX_LENGTH)


data_encodings = dict_dataset.map(tokenize, batched=True)
data_encodings

Map:   0%|          | 0/1869 [00:00<?, ? examples/s]

Map:   0%|          | 0/272 [00:00<?, ? examples/s]

Map:   0%|          | 0/530 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Label', 'Text', 'input_ids', 'attention_mask'],
        num_rows: 1869
    })
    val: Dataset({
        features: ['Label', 'Text', 'input_ids', 'attention_mask'],
        num_rows: 272
    })
    test: Dataset({
        features: ['Label', 'Text', 'input_ids', 'attention_mask'],
        num_rows: 530
    })
})

# Modelo

Necesitamos hacer algunas modificaciones para preparar el dataset para el modelo:

- Eliminamos la columna 'text', porque es un campo que el modelo no espera.
- Renombramos 'label' a 'labels', porque es el nombre que espera el modelo.
- El dataset debe devolver un objeto Tensor en lugar de una lista.

Para pasarle los datos al modelo debemos guardarlos en objetos DataLoader

In [ ]:
data_encodings = data_encodings.remove_columns('Text')
data_encodings = data_encodings.rename_column('Label', 'labels')
data_encodings.set_format("torch")
data_encodings

train_dataloader = DataLoader(data_encodings['train'], shuffle=True, batch_size=8)
val_dataloader = DataLoader(data_encodings['val'], batch_size=8)

Para definir el modelo, hay que establecer el número de etiquetas:

In [ ]:
NUM_LABELS = 2
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=NUM_LABELS) 

Some weights of the model checkpoint at dccuchile/distilbert-base-spanish-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at dccuchile/distilbert-base-spanish-uncased and are newly initialized: ['classifier.weight'

Como optimizador vamos a utilizar el optijizador Adam, implementado en AdamW, y definimos el learning rate:

In [ ]:
optimizer = AdamW(model.parameters(), lr=5e-5)

# Hyperparámetros

In [ ]:
#!pip uninstall -y transformers accelerate
#!pip install transformers accelerate

In [ ]:
args = TrainingArguments(output_dir="./outputs")
args.evaluation_strategy="epoch"
args.per_device_train_batch_size = 32
args.per_device_eval_batch_size = 32
args.num_train_epochs = 5

# Métrica

In [ ]:
def compute_metrics(pred):
    """recibe un lote prediciones inferidas por el modelo. """
    y_true = pred.label_ids # son las labels en el gold standard
    y_pred = pred.predictions.argmax(-1) # pred.predictions devuelve una lista con las predicciones
                                         # para cada clase. Debemos quedarnos con la de mayor probabilidad.

    # como son varias clases, utilizaremos la macro
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')
    acc = accuracy_score(y_true, y_pred)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Training

In [ ]:
trainer = Trainer(
    model=model,            # modelo que será ajustado
    args = args,     # hiperparámetros
    train_dataset=data_encodings['train'], # conjunto training
    eval_dataset=data_encodings['val'],   # conjunto de validación
    compute_metrics=compute_metrics,    # función para computar las métricas
)


In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.164636,0.944853,0.922959,0.916276,0.930184
2,No log,0.111566,0.970588,0.957233,0.968740,0.946851
3,No log,0.149908,0.963235,0.947168,0.952516,0.942089
4,No log,0.142557,0.966912,0.952172,0.960492,0.944470
5,No log,0.144295,0.970588,0.957233,0.968740,0.946851


TrainOutput(global_step=295, training_loss=0.12633288108696372, metrics={'train_runtime': 51.8596, 'train_samples_per_second': 180.198, 'train_steps_per_second': 5.688, 'total_flos': 132978381297300.0, 'train_loss': 0.12633288108696372, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.1442945897579193,
 'eval_accuracy': 0.9705882352941176,
 'eval_f1': 0.9572327044025157,
 'eval_precision': 0.9687399291008701,
 'eval_recall': 0.9468509984639017,
 'eval_runtime': 0.4318,
 'eval_samples_per_second': 629.911,
 'eval_steps_per_second': 20.843,
 'epoch': 5.0}

# Evaluación

In [ ]:
def get_prediction(text):
    # prepara el texto, aplicamos la misma tokenización que la utilizada en el training
    inputs = tokenizer(text, padding="max_length", truncation=True, return_tensors="pt").to("cuda")
    # aplicamos el modelo
    outputs = model(**inputs)
    # obtenemos la probabilidad para cada clase
    probs = outputs[0].softmax(1)
    # argmax nos devuelve la clase con mayor probabilidad.
    # argmax devuelve un tensor. Debemos devolver su valor asociado 
    return probs.argmax().item()

In [ ]:
y_pred=[get_prediction(text) for text in dict_dataset['test']['Text']]
print(classification_report(y_true=dict_dataset['test']['Label'], y_pred=y_pred, target_names=['0', '1']))

              precision    recall  f1-score   support

           0       0.95      0.97      0.96       410
           1       0.88      0.82      0.85       120

    accuracy                           0.93       530
   macro avg       0.92      0.89      0.90       530
weighted avg       0.93      0.93      0.93       530

